### Helper code to match memcat images names to mooney image names

In [22]:
import hashlib
#from scipy.misc import imread, imresize, imshow
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
import numpy as np
import os
from imageio import imread, imsave
import pandas as pd

### Flatten dir structure

In [6]:
import os
import sys
import string
import shutil

#Generate the file paths to traverse, or a single path if a file name was given
def getfiles(path):
    if os.path.isdir(path):
        for root, dirs, files in os.walk(path):
            for name in files:
                yield os.path.join(root, name)
    else:
        yield path



In [9]:
destination = "/home/sander/Downloads/memcats/"
fromdir = "/home/sander/Downloads/MemCat"
for f in getfiles(fromdir):
    filename = str.split(f, '/')[-1]
    if os.path.isfile(destination+filename):
        filename = f.replace(fromdir,"",1).replace("/","_")
        print('naming conflict')
    #os.rename(f, destination+filename)
    shutil.copy(f, destination+filename)

In [24]:
os.chdir(r'/home/sander/Downloads/gray')
file_list = os.listdir()
print(len(file_list))

8721


### Find duplicates based on file hash and save matching names

In [26]:
duplicates = []
matchlist = []
hash_keys = dict()
for index, filename in enumerate(file_list):  #listdir('.') = current directory
    if os.path.isfile(filename):
        with open(filename, 'rb') as f:
            filehash = hashlib.md5(f.read()).hexdigest()
        if filehash not in hash_keys: 
            hash_keys[filehash] = index
        else:
            duplicates.append((index, filename, hash_keys[filehash]))
            if 'gs' in filename:
                matchlist.append((filename, file_list[hash_keys[filehash]]))
            else: matchlist.append((file_list[hash_keys[filehash]], filename))

In [28]:
df = pd.DataFrame(matchlist, columns =['mooneyname', 'memcatname'])

df.to_csv('matchedimgnames.csv')

In [9]:

def rgb2gray(img):
    """ Given an RGB image return the gray scale image.
    Based on http://en.wikipedia.org/wiki/Grayscale#Converting_color_to_grayscale
    img = 0.299 R + 0.587 G + 0.114 B
    """
    print('Converting RGB image to gray scale.')
    return np.uint8(np.dot(img[...,:3], [0.299, 0.587, 0.114]))



In [ ]:


for img in file_list:
    im = imread(img)
    im = rgb2gray(im)
    filename = os.path.join('gray', img)
    imsave(filename, im.astype('float'))

